Vector Search Qdrant

## 1. Importación de librerías y conexión con Qdrant

Primero asegúrate de correr e instalar las dependencias necesarias
Y dejar el contenedor corriendo. <p>
Setup: [set_up_qdrant]()

Importación de librerías
- Clase `QdrantClient`: permite establecer conexión con el servicio de Qdrant.
- Módulo `models`: permite establecer configuraciones y parámetros necesarios.

In [4]:
from qdrant_client import QdrantClient, models

In [5]:
# Inicializamos el cliente
client = QdrantClient("http://localhost:6333")

## 2. Dataset


Hay que entender el dataset, conocer principalmente:
- Qué tipo de contenido es? imagen, video, texto o combinación
- Especificar: Si es texto, entonces qué lenguaje es, contiene caracteres especiales?

¿Porqué es necesario responder estas preguntas?
Para definir:
- El tipo de esquema para los datos. (¿qué es lo se va a vectorizar?, ¿Qué se va a almacenar en la metadata?)
- El modelo correcto del embedding. Tenemos que evaluar diferentes parámetros como el dominio, la precisión y los recursos requeridos           

In [ ]:
# Importación del dataset
# pip install requests
import requests

ruta_doc= 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
docs_rpta= requests.get(ruta_doc)
documentos = docs_rpta.json()
# documentos

In [ ]:
print(f"Tamaño de documentos:{len(documentos)}")
print(f"Data type de documentos:{type(documentos)}")
print(f"Data type de dato del primer elemento de documentos:{type(documentos[0])}")

print('\nCursos del FaQ Zoomcamp:')
for i in documentos:
    print(f"- {i['course']}")

Tamaño de documentos:3
Data type de documentos:<class 'list'>
Data type de dato del primer elemento de documentos:<class 'dict'>

Cursos del FaQ Zoomcamp:
- data-engineering-zoomcamp
- machine-learning-zoomcamp
- mlops-zoomcamp


Entonces, después de analizar `documentos`, la información relevante es:
- Tamaño: 3 elementos
- Lista de diccionarios

Estructura de `documentos` <p>

```git bash
[
    {'course':'data-engineering-zoomcamp',
    'documents': [{'text': ' ' , 'section':' '  , 'question':' '  },
                  {'text': ' ' , 'section':' '  , 'question':' '  },
                                 ...                    
                {'text':  , 'section':  , 'question':  } ]
    
    }, 
    {'course':'machine-learning-zoomcamp',
    'documents': [{'text':  , 'section':  , 'question':  },
                  {'text':  , 'section':  , 'question':  },
                                 ...                    
                {'text':  , 'section':  , 'question':  } ]
    
    }, 
    {'course':'mlops-zoomcamp',
    'documents': [{'text':  , 'section':  , 'question':  },
                  {'text':  , 'section':  , 'question':  },
                                 ...                    
                {'text':  , 'section':  , 'question':  } ]
    } 
]
```

Cada elemento de la lista es un diccionario, que contiene 2 elementos(course y documents). <p>
Del cual documents es también una lista de diccionarios, donde cada diccionario contiene 3 elementos(text, section, question).

1. Verificamos que la data se encuentra limpia y fragmentada (chunk, divide la data en pequeñas partes porque es más fácil para los modelos de embeddings procesarla)
2. Tenemos que definir
    - Los campos(fields) para semantic_search
    - Campos para almacenar como *metadata*, pueden ser usados como filtros 

3. Se definen: 
    - Campos para semantic search: question, text
    - Campos para metadata(también como filtros): course, section

## 3. Elección del modelo de embeddings

Los modelos de embeddings como ya lo hemos mencionado permite convertir la data en vectores. Para escoger un 'buen' modelo, depende ciertos factores:
- La tarea, el tipo de dato y sus características. (texto, inglés )
- La evaluación entre la 'precisión de búsqueda' y los 'recursos usados' (embeddings más grandes requieren más almacenamiento y memoria)
- El costo de deducir(o inferir) 
etc

La mejor manera de escoger el modelo es testear diferentes opciones en tu propia data. <p>
En este caso, vamos a utilizar FastEmbed. <p>
Documentación: [FastEmbed](https://github.com/qdrant/fastembed).


In [ ]:
from fastembed import TextEmbedding

# modelos disponibles
modelos= TextEmbedding.list_supported_models()

# primeros 5 modelos
modelos[:5] 

# modelo_embedding= TextEmbedding()

[{'model': 'BAAI/bge-base-en',
  'sources': {'hf': 'Qdrant/fast-bge-base-en',
   'url': 'https://storage.googleapis.com/qdrant-fastembed/fast-bge-base-en.tar.gz',
   '_deprecated_tar_struct': True},
  'model_file': 'model_optimized.onnx',
  'description': 'Text embeddings, Unimodal (text), English, 512 input tokens truncation, Prefixes for queries/documents: necessary, 2023 year.',
  'license': 'mit',
  'size_in_GB': 0.42,
  'additional_files': [],
  'dim': 768,
  'tasks': {}},
 {'model': 'BAAI/bge-base-en-v1.5',
  'sources': {'hf': 'qdrant/bge-base-en-v1.5-onnx-q',
   'url': 'https://storage.googleapis.com/qdrant-fastembed/fast-bge-base-en-v1.5.tar.gz',
   '_deprecated_tar_struct': True},
  'model_file': 'model_optimized.onnx',
  'description': 'Text embeddings, Unimodal (text), English, 512 input tokens truncation, Prefixes for queries/documents: not so necessary, 2023 year.',
  'license': 'mit',
  'size_in_GB': 0.21,
  'additional_files': [],
  'dim': 768,
  'tasks': {}},
 {'model':

Es lógico que nuestro modelo no produzca una alta dimensionalidad para evitar usar recursos de más, por lo tanto nos declinamos por una moderada dimensionalidad (Ej: 512 dimensionalidades)

In [27]:
import json

In [ ]:
# para formatear la salida con indentacion
import json

dimension= 512
for i in modelos:
    if i['dim'] == dimension:
        print (json.dumps(i, indent=2))



{
  "model": "BAAI/bge-small-zh-v1.5",
  "sources": {
    "hf": "Qdrant/bge-small-zh-v1.5",
    "url": "https://storage.googleapis.com/qdrant-fastembed/fast-bge-small-zh-v1.5.tar.gz",
    "_deprecated_tar_struct": true
  },
  "model_file": "model_optimized.onnx",
  "description": "Text embeddings, Unimodal (text), Chinese, 512 input tokens truncation, Prefixes for queries/documents: not so necessary, 2023 year.",
  "license": "mit",
  "size_in_GB": 0.09,
  "additional_files": [],
  "dim": 512,
  "tasks": {}
}
{
  "model": "Qdrant/clip-ViT-B-32-text",
  "sources": {
    "hf": "Qdrant/clip-ViT-B-32-text",
    "url": null,
    "_deprecated_tar_struct": false
  },
  "model_file": "model.onnx",
  "description": "Text embeddings, Multimodal (text&image), English, 77 input tokens truncation, Prefixes for queries/documents: not necessary, 2021 year",
  "license": "mit",
  "size_in_GB": 0.25,
  "additional_files": [],
  "dim": 512,
  "tasks": {}
}
{
  "model": "jinaai/jina-embeddings-v2-small-e

El modelo 1: `BAAI/bge-small-zh-v1.5`
- Para texto pero para lenguaje Chino, descartado

El modelo 2: `Qdrant/clip-ViT-B-32-text`
- Multimoda, Para texto e imágenes, pero no es necesario impágenes, descartado

El modelo 3: `jinaai/jina-embeddings-v2-small-en`
- El más adecuado descartando los previos

In [ ]:
modelo_seleccionado= "jinaai/jina-embeddings-v2-small-en"

# Este modelo como muchos, fue entrenado usando similitud de coseno


In [ ]:
# ejemplo fastembed
from fastembed import TextEmbedding


# Example list of documents
documents: list[str] = [
    "This is built to be faster and lighter than other embedding libraries e.g. Transformers, Sentence-Transformers, etc.",
    "fastembed is supported by and maintained by Qdrant.",
]

# This will trigger the model download and initialization
embedding_model = TextEmbedding()
print("The model BAAI/bge-small-en-v1.5 is ready to use.")

embeddings_generator = embedding_model.embed(documents)  # reminder this is a generator
embeddings_list = list(embedding_model.embed(documents))
  # you can also convert the generator to a list, and that to a numpy array
len(embeddings_list[0]) # Vector of 384 dimensions

In [10]:

documentos[0]['course']

'data-engineering-zoomcamp'